In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    ORDER BY MEDV
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,38.35180,0.0,18.1,0.0,0.693,5.453,100.0,1.4896,24.0,666.0,20.2,396.90,30.59,5.0
1,67.92080,0.0,18.1,0.0,0.693,5.683,100.0,1.4254,24.0,666.0,20.2,384.97,22.98,5.0
2,25.04610,0.0,18.1,0.0,0.693,5.987,100.0,1.5888,24.0,666.0,20.2,396.90,26.77,5.6
3,9.91655,0.0,18.1,0.0,0.693,5.852,77.8,1.5004,24.0,666.0,20.2,338.16,29.97,6.3
4,45.74610,0.0,18.1,0.0,0.693,4.519,100.0,1.6582,24.0,666.0,20.2,88.27,36.98,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,4.89822,0.0,18.1,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26,50.0
502,5.66998,0.0,18.1,1.0,0.631,6.683,96.8,1.3567,24.0,666.0,20.2,375.33,3.73,50.0
503,6.53876,0.0,18.1,1.0,0.631,7.016,97.5,1.2024,24.0,666.0,20.2,392.05,2.96,50.0
504,9.23230,0.0,18.1,0.0,0.631,6.216,100.0,1.1691,24.0,666.0,20.2,366.15,9.53,50.0


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
# Функция COUNT возвращает количество элементов не равных NULL. 
# Значит, можно отнять количество непустых значений от общего количества строк
tasks+=1
pd.read_sql(
    '''
    SELECT 
        COUNT(*) - COUNT(CRIM) AS CRIM, 
        COUNT(*) - COUNT(ZN) AS ZN, 
        COUNT(*) - COUNT(INDUS) AS INDUS, 
        COUNT(*) - COUNT(CHAS) AS CHAS, 
        COUNT(*) - COUNT(NOX) AS NOX
    FROM boston
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,0,0,0,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT 
        COUNT(DISTINCT CRIM)  AS CRIM,
        COUNT(DISTINCT ZN) AS ZN,
        COUNT(DISTINCT INDUS) AS INDUS,
        COUNT(DISTINCT CHAS) AS CHAS,
        COUNT(DISTINCT NOX) AS NOX
    FROM boston;
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,26,76,2,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [9]:
# Вероятно, здесь есть более простое решение, 
# но я его не нашел за две недели, 
# поэтому представляю то, что смог родить
tasks+=1
pd.read_sql(
    """
    SELECT g.MEAN_CRIM, g.MEAN_ZN, g.MEAN_INDUS, g.MEAN_CHAS, h.MEAN_NOX FROM
        (SELECT e.MEAN_CRIM, e.MEAN_ZN, e.MEAN_INDUS, f.MEAN_CHAS, e.id FROM
            (SELECT c.MEAN_CRIM, c.MEAN_ZN, d.MEAN_INDUS, c.id AS id FROM
                (SELECT a.MEAN_CRIM AS MEAN_CRIM, b.MEAN_ZN AS MEAN_ZN, a.id AS id FROM
                    (SELECT SUM(CRIM) / 2 AS MEAN_CRIM, id FROM (SELECT CRIM, ROW_NUMBER() OVER (ORDER BY CRIM) AS id FROM boston LIMIT 253, 2)) a
                    LEFT JOIN
                    (SELECT SUM(ZN) / 2 AS MEAN_ZN, id FROM (SELECT ZN, ROW_NUMBER() OVER (ORDER BY ZN) AS id FROM boston LIMIT 253, 2)) b
                    ON a.id = b.id) c
            LEFT JOIN
            (SELECT SUM(INDUS) / 2 AS MEAN_INDUS, id FROM (SELECT INDUS, ROW_NUMBER() OVER (ORDER BY INDUS) AS id FROM boston LIMIT 253, 2)) d
            ON c.id = d.id) e
        LEFT JOIN
        (SELECT SUM(CHAS) / 2 AS MEAN_CHAS, id FROM (SELECT CHAS, ROW_NUMBER() OVER (ORDER BY CHAS) AS id FROM boston LIMIT 253, 2)) f
        ON e.id = f.id) g
    LEFT JOIN
    (SELECT SUM(NOX) / 2 AS MEAN_NOX, id FROM (SELECT NOX, ROW_NUMBER() OVER (ORDER BY NOX) AS id FROM boston LIMIT 253, 2)) h
    ON g.id = h.id
    """,
    con,
)

,MEAN_CRIM,MEAN_ZN,MEAN_INDUS,MEAN_CHAS,MEAN_NOX
0,0.26042,0.0,9.69,0.0,0.538


Очевидно, что минимальные значения у колонок MEAN_ZN и MEAN_CHAS

ZN - доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.
CHAS - 1, если граничит с рекой, 0 иначе

Если вывести только названия этих полей, то не будет понятно, какой вывод делать, а с данными можно сделать вывод:
- более половины всех земель под жилую застройку по площади менее 25000 кв. футов
- более половины земель не граничит с рекой

4. Выведите разницу между средним количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [10]:
tasks+=1
pd.read_sql(
    """
    with rooms as (
    SELECT 
            row_number() OVER () AS CHEAP_HOUSES_COUNT, 
            AVG(a.RM) OVER (ORDER BY MEDV ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS AVG_RM_CHEAP,
            b.AVG_RM_EXP,
            b.AVG_RM_EXP - AVG(a.RM) OVER (ORDER BY MEDV ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS DIFF
        FROM boston a
        LEFT JOIN
        (SELECT DISTINCT AVG(RM) OVER(PARTITION BY MEDV) AS AVG_RM_EXP FROM boston WHERE MEDV = 50) b
    ) 
    select * from rooms where CHEAP_HOUSES_COUNT = 25
    union all
    select * from rooms where CHEAP_HOUSES_COUNT = 50
    union all
    select * from rooms where CHEAP_HOUSES_COUNT = 100
    union all
    select * from rooms where CHEAP_HOUSES_COUNT = 200
    union all
    select * from rooms where CHEAP_HOUSES_COUNT = 300
    """,
    con,
)

,CHEAP_HOUSES_COUNT,AVG_RM_CHEAP,AVG_RM_EXP,DIFF
0,25,5.747840,7.484,1.736160
1,50,5.753240,7.484,1.730760
2,100,5.887120,7.484,1.596880
3,200,5.911705,7.484,1.572295
4,300,5.972227,7.484,1.511773


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [11]:
tasks+=1
pd.read_sql(
    '''
    SELECT LSTAT, MEDV, rank_by_medv FROM
        (SELECT 
            LSTAT, 
            MEDV, 
            rank() OVER (PARTITION BY MEDV ORDER BY LSTAT) rank_by_medv, 
            MAX(MEDV) OVER() AS MAX_MEDV
        FROM boston)
    WHERE MEDV = MAX_MEDV
    ''',
    con,
)

,LSTAT,MEDV,rank_by_medv
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,3
3,2.96,50.0,4
4,2.97,50.0,5
5,3.16,50.0,6
6,3.26,50.0,7
7,3.32,50.0,8
8,3.70,50.0,9
9,3.73,50.0,10


Вывод: В дорогих домах проживает не более 10% населения с низким статусом

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT DISTINCT CHAS AS "Граничит/не граничит", AVG(MEDV) OVER(PARTITION BY CHAS) AS "Средняя стоимость" 
    FROM boston
    ''',
    con,
)

,Граничит/не граничит,Средняя стоимость
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [13]:
# Вероятно, здесь есть более простое решение, 
# но я его не нашел за две недели, 
# поэтому представляю то, что смог родить
tasks+=1
pd.read_sql(
    '''
    SELECT g.AVG_NOX, g.AVG_CRIM, g.AVG_ZN, g.AVG_INDUS, h.AVG_CHAS FROM
        (SELECT e."Граничит/не граничит", e.AVG_NOX, e.AVG_CRIM, e.AVG_ZN, f.AVG_INDUS FROM
            (SELECT c."Граничит/не граничит", c.AVG_NOX, c.AVG_CRIM, d.AVG_ZN FROM
                (SELECT a."Граничит/не граничит", a.AVG_NOX, b.AVG_CRIM FROM
                    (SELECT * FROM
                        (SELECT *, MAX(AVG_NOX) OVER() AS MAX_AVG_NOX FROM
                            (SELECT 
                                DISTINCT CHAS AS "Граничит/не граничит", 
                                AVG(NOX) OVER(PARTITION BY CHAS) AS AVG_NOX 
                            FROM boston))
                    WHERE AVG_NOX = MAX_AVG_NOX AND "Граничит/не граничит" = 1) a
                    LEFT JOIN
                    (SELECT * FROM
                        (SELECT *, MAX(AVG_CRIM) OVER() AS MAX_AVG_CRIM FROM
                            (SELECT 
                                DISTINCT CHAS AS "Граничит/не граничит", 
                                AVG(CRIM) OVER(PARTITION BY CHAS) AS AVG_CRIM 
                            FROM boston))
                    WHERE AVG_CRIM = MAX_AVG_CRIM AND "Граничит/не граничит" = 1) b
                ON a."Граничит/не граничит" = b."Граничит/не граничит") c
            LEFT JOIN
                (SELECT * FROM
                    (SELECT *, MAX(AVG_ZN) OVER() AS MAX_AVG_ZN FROM
                        (SELECT 
                            DISTINCT CHAS AS "Граничит/не граничит", 
                            AVG(ZN) OVER(PARTITION BY CHAS) AS AVG_ZN 
                        FROM boston))
                WHERE AVG_ZN = MAX_AVG_ZN AND "Граничит/не граничит" = 1) d
            ON c."Граничит/не граничит" = d."Граничит/не граничит") e
        LEFT JOIN
            (SELECT * FROM
                (SELECT *, MAX(AVG_INDUS) OVER() AS MAX_AVG_INDUS FROM
                    (SELECT 
                        DISTINCT CHAS AS "Граничит/не граничит", 
                        AVG(INDUS) OVER(PARTITION BY CHAS) AS AVG_INDUS 
                    FROM boston))
            WHERE AVG_INDUS = MAX_AVG_INDUS AND "Граничит/не граничит" = 1) f
        ON e."Граничит/не граничит" = f."Граничит/не граничит") g
    LEFT JOIN
        (SELECT * FROM
            (SELECT *, MAX(AVG_CHAS) OVER() AS MAX_AVG_CHAS FROM
                (SELECT 
                    DISTINCT CHAS AS "Граничит/не граничит", 
                    AVG(CHAS) OVER(PARTITION BY CHAS) AS AVG_CHAS 
                FROM boston))
            WHERE AVG_CHAS = MAX_AVG_CHAS AND "Граничит/не граничит" = 1) h
    ON g."Граничит/не граничит" = h."Граничит/не граничит"
    ''',
    con,
)

,AVG_NOX,AVG_CRIM,AVG_ZN,AVG_INDUS,AVG_CHAS
0,0.593426,None,None,12.719143,1.0


Вывод:
- рядом с рекой концентрация оксида озота выше
- уровень преступности на душу населения рядом с рекой меньше
- доля акров неторгового бизнеса на город рядом с рекой выше
- рядом с рекой меньше земель зонировано под большие участки (более 25000 кв. футов), чем вдали от реки
- про показатель chas ничего нельзя сказать - понятно, что 1 > 0

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [14]:

tasks+=1
pd.read_sql(
    '''
    SELECT a.PERCENTS, a.NOX, b.INDUS FROM
        (SELECT * FROM
            (SELECT *, ROW_NUMBER() OVER(PARTITION BY PERCENTS) AS R_NUM FROM
                    (SELECT *, CAST(COUNT*100/CAST(TOTAL AS FLOAT) AS INT) AS PERCENTS FROM
                        (SELECT NOX, ROW_NUMBER() OVER (ORDER BY NOX) AS COUNT, COUNT() OVER() AS TOTAL FROM boston)
                    WHERE PERCENTS % 10 = 0))
        WHERE R_NUM = 1) a
    LEFT JOIN
        (SELECT * FROM
            (SELECT *, ROW_NUMBER() OVER(PARTITION BY PERCENTS) AS R_NUM FROM
                    (SELECT *, CAST(COUNT*100/CAST(TOTAL AS FLOAT) AS INT) AS PERCENTS FROM
                        (SELECT INDUS, ROW_NUMBER() OVER (ORDER BY INDUS) AS COUNT, COUNT() OVER() AS TOTAL FROM boston)
                    WHERE PERCENTS % 10 = 0))
        WHERE R_NUM = 1) b
    ON a.PERCENTS = b.PERCENTS
    ''',
    con,
)

,PERCENTS,NOX,INDUS
0,0,0.385,0.46
1,10,0.426,2.89
2,20,0.442,4.39
3,30,0.472,5.96
4,40,0.507,7.38
5,50,0.538,9.69
6,60,0.575,12.83
7,70,0.605,18.10
8,80,0.668,18.10
9,90,0.713,19.58


Взаимосвязи между "долей акров неторгового бизнеса на город" и "концентрация оксидов азота (частей на 10 миллионов)" не наблюдается

In [15]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
